This notebook is based on the file mimic_dataset_labels_creation.py and obtains the labels for diseases and drugs from the MIMIC dataset.

Ensure the following files from MIMIC3 are available in the present directory:
* PRESCRIPTIONS.csv
* nsides_rxcui_icd9.csv
* ndc_map 2020_06_17 (atc5 atc4 ingredients).csv
* DIAGNOSES_ICD.csv

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
# step 1
#load MIMIC3
dname = "./"
dname_nsides = "./"

fname_prescrip = dname + "PRESCRIPTIONS.csv"
fname_drugs = dname_nsides + "nsides_rxcui_icd9.csv"
fname_map = "./ndc_map 2020_06_17 (atc5 atc4 ingredients).csv"
fname_diagnosis = dname + "DIAGNOSES_ICD.csv"

df_pres = pd.read_csv(fname_prescrip,dtype={"NDC":str})
df_drugs = pd.read_csv(fname_drugs,dtype={"RXCUI":str,"ICD9_CODE":str})
df_map = pd.read_csv(fname_map,dtype={"in_rxcui":str})
df_diag = pd.read_csv(fname_diagnosis)


In [4]:
#from: https://github.com/chb/py-umls/blob/d4ba79245b146c1a59363668d0ef4918689c25d1/rxnorm.py#L45
def ndc_normalize(ndc):
    """ Normalizes an NDC (National Drug Code) number.

    The pseudo-code published by NIH
    (http://www.nlm.nih.gov/research/umls/rxnorm/NDC_Normalization_Code.rtf)
    first identifies the format (e.g. "6-3-2") and then normalizes based on
    that finding. However since the normalized string is always 5-4-2,
    padded with leading zeroes and removing all dashes afterwards, this
    implementation goes a much simpler route.

    NDCs that only contain one dash are treated as if they were missing the
    package specifier, so they get a "-00" appended before normalization.

    :param str ndc: The NDC to normalize as string
    :returns: A string with the normalized NDC, or `None` if the number
        couldn't be normalized
    """
    if ndc is None or 0 == len(ndc) or len(ndc) > 14:
        return None

    # replace '*' with '0' as some of the NDCs from MTHFDA contain * instead of 0
    norm = ndc.replace('*', '0')

    # split at dashes, pad with leading zeroes, cut to desired length
    parts = norm.split('-')

    # Code with only one dash; this is NOT mentioned in the above cited
    # reference but I see a lot of codes with 6-4 format.
    # These are likely codes without package specifier, though some that I
    # checked seem to not or no longer exist.
    # We append "-00" to get a 6-4-2 format and are done with it.
    if 2 == len(parts):
        parts.append('00')

    # two dashes, 6-4-1 or 5-3-2 or similar formats, concat to 5-4-2
    if 3 == len(parts):
        norm = '{}{}{}'.format(('00000'+parts[0])[-5:], ('0000'+parts[1])[-4:], ('00'+parts[2])[-2:])

    # no dashes
    elif 1 == len(parts):

        # "if NDC passed has 12 digits and first char is '0' and it's from
        # VANDF then trim first char". We do NOT check if it's from the VA
        # as this would require more information than just the NDC
        if 12 == len(norm) and '0' == norm[:1]:
            norm = norm[1:]

        # only valid if it's 11 digits
        elif 11 != len(norm):
            return None

    # reject NDCs that still contain non-numeric chars
    return norm if norm.isdigit() else None

##################

In [5]:
#Add ndc_norm to external data "df_map" and subset only the reqd columns

list_ndc_norm = []
for idx,row in df_map.iterrows():
    cur_ndc = row["ndc"]
    cur_ndc_norm = ndc_normalize(cur_ndc)
    list_ndc_norm.append(cur_ndc_norm)

df_map["NDC_NORM"] = list_ndc_norm

df_map_cols_subset = df_map[["in_rxcui","NDC_NORM"]].drop_duplicates()


#Add ndc_norm to MIMIC prescriptions "df_pres" and subset only the reqd columns

df_pres = pd.merge(left=df_pres,right=df_map_cols_subset,\
                          how='left',left_on="NDC",right_on="NDC_NORM").drop_duplicates()


In [6]:
###################
# Drugs
###################

#step 2
#our dataset
#TODO: replace with the version of data pkl file used in our experiment
fname_data = "./dict_nsides_mimic_data_v1_case1_part.pkl"
data = pkl.load(open(fname_data,"rb"))
#
list_drugs_id_rxcui = list(data["metadata"]["dict_r_id_idx_map"].keys())
list_drugs_id_rxcui_num = [float(cur_num) for cur_num in list_drugs_id_rxcui]
#
list_pat_id = list(data["metadata"]["dict_p_id_idx_map"].keys())
list_d_id = list(data["metadata"]["dict_d_id_idx_map"].keys())


In [7]:
# confirm sizes against what was used in NCMF
print(len(list_d_id))
print(len(list_pat_id))
print(len(list_drugs_id_rxcui))

1321
5891
596


In [8]:
#step 3
# Find the subset of MIMIC containing our dataset's patients and drugs
df_pres_subset = df_pres[(df_pres["in_rxcui"].isin(list_drugs_id_rxcui))&(df_pres["SUBJECT_ID"].isin(list_pat_id))]


In [9]:
df_pres_subset.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,...,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,in_rxcui,NDC_NORM
638,2982464,11,194540,229441.0,2178-04-16 00:00:00,2178-04-21 00:00:00,MAIN,Dexamethasone,Dexamethasone,Dexamethasone,...,006789,00054418425,4mg Tab,4,mg,1,TAB,PO,3264,00054418425
640,2982467,11,194540,229441.0,2178-04-16 00:00:00,2178-04-22 00:00:00,MAIN,Phenytoin,Phenytoin,Phenytoin Sodium Extended,...,004521,51079090520,100mg Capsule,100,mg,1,CAP,PO,8183,51079090520
647,2981780,11,194540,229441.0,2178-04-17 00:00:00,2178-04-17 00:00:00,BASE,Potassium Chl 20 mEq / 1000 mL D5 1/2 NS,NaN,NaN,...,002003,00338067104,1000 mL Bag,1000,ml,1,BAG,IV,4850,00338067104
649,2981780,11,194540,229441.0,2178-04-17 00:00:00,2178-04-17 00:00:00,BASE,Potassium Chl 20 mEq / 1000 mL D5 1/2 NS,NaN,NaN,...,002003,00338067104,1000 mL Bag,1000,ml,1,BAG,IV,8591,00338067104
650,2981780,11,194540,229441.0,2178-04-17 00:00:00,2178-04-17 00:00:00,BASE,Potassium Chl 20 mEq / 1000 mL D5 1/2 NS,NaN,NaN,...,002003,00338067104,1000 mL Bag,1000,ml,1,BAG,IV,9863,00338067104


In [10]:
df_pres_subset.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE',
       'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
       'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX',
       'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE', 'in_rxcui',
       'NDC_NORM'],
      dtype='object')

In [11]:
len(df_pres_subset.in_rxcui.unique())

538

In [12]:
len(df_pres_subset.SUBJECT_ID.unique())

5891

In [13]:
df_pres_subset.shape

(540856, 21)

In [14]:
# drugs with embeddings available
available_drugs = df_pres_subset["in_rxcui"].unique()

In [15]:
# df_pres_subset.to_csv("MIMIC_drug_patient_info.csv")

In [16]:
len(available_drugs)

538

In [17]:
len(set(list_drugs_id_rxcui) - set(available_drugs))

58

In [18]:
#step 4
# Use the above dataframe to find a drug's DRUG_TYPE / ROUTE for each (SUBJECT_ID and DRUG) pair in our dataset
# Select a drug metadata as label
# Potential drug labels
df_pres_subset["DRUG_TYPE"].unique()


array(['MAIN', 'BASE', 'ADDITIVE'], dtype=object)

In [19]:
df_pres_subset["ROUTE"].unique()

array(['PO', 'IV', 'PR', 'IV DRIP', 'IH', 'OU', 'PB', 'TP', 'SC', 'ED',
       'NU', 'IM', 'ET', 'TD', 'IR', 'DWELL', 'VG', 'LEFT EYE', 'NG',
       'BOTH EYES', 'ID', 'PO/NG', 'DIALYS', 'NEB', 'ORAL', 'IV BOLUS',
       'LOCK', 'IT', 'OD', 'IN', 'OS', 'SL', 'NAS', 'IRR', 'NS', 'G TUBE',
       'IP', 'PL', 'BU', 'BOTH EARS', 'AU', 'IVPCA', 'RIGHT EYE', 'IJ',
       'TT', 'SUBCUT', 'J TUBE', 'AD', 'RIGHT EAR', 'BUCCAL', 'LEFT EAR',
       'IA', 'AS', 'OG', nan, 'EX-VIVO', 'ND', 'NG/OG', 'INHALATION',
       'INTRAPERICARDIAL', 'PERIPHNERVE', 'PO/OG', 'IVT', 'INTERSPACE',
       'REPLACE'], dtype=object)

In [20]:
###################
# Disease
###################

df_diag_subset = df_diag[df_diag["SUBJECT_ID"].isin(list_pat_id) & df_diag["ICD9_CODE"].isin(list_d_id)]

#df_diag_subset2 = df_diag_subset[df_diag_subset[""].isin([117,124])]

#potential labels for diseases
df_diag_subset["SEQ_NUM"]


1          2.0
4          5.0
6          7.0
8          9.0
10        11.0
          ... 
650949    21.0
650950    22.0
650953    25.0
650955    27.0
650958    30.0
Name: SEQ_NUM, Length: 34754, dtype: float64

In [21]:
df_diag_subset.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
1,1298,109,172335,2.0,486
4,1301,109,172335,5.0,4254
6,1303,109,172335,7.0,7100
8,1305,109,172335,9.0,7243
10,1307,109,172335,11.0,2875


In [22]:
len(df_diag_subset.ICD9_CODE.unique())

907

In [23]:
len(df_diag_subset["SEQ_NUM"].unique())

39

In [24]:
len(df_diag_subset["SUBJECT_ID"].unique())

5677

In [25]:
# df_diag_subset.to_csv("MIMIC_drug_disease_info.csv")